<a href="https://colab.research.google.com/github/haoruilee/js_competitions/blob/kaggle-upload-1/js-transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import os
import statistics as stat
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_squared_error
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/jane-street-real-time-market-data-forecasting

features.csv	   lags.parquet    sample_submission.csv  train.parquet
kaggle_evaluation  responders.csv  test.parquet


In [ ]:
!ls ./drive/jane-street-real-time-market-data-forecasting

ls: cannot access './drive/jane-street-real-time-market-data-forecasting': No such file or directory


In [3]:

jane_street_real_time_market_data_forecasting_path = "./drive/MyDrive/jane-street-real-time-market-data-forecasting"  # Set your path here
checkpoint_dir = "./drive/MyDrive/kaggle_js_checkpoints_transformer"


In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import polars as pl
import os
import pandas as pd

valid_from = 0

# Load data
start_time = time.time()
print("Loading data...")
alltraindata = pl.scan_parquet(f"{jane_street_real_time_market_data_forecasting_path}/train.parquet")
train = alltraindata.filter(pl.col("date_id") >= valid_from).collect()
print(f"Data loaded in {time.time() - start_time:.2f} seconds")

# Prepare features
start_time = time.time()
print("Preparing features...")
feature_names = [f"feature_{i:02d}" for i in range(79)]
train_features = train.select(feature_names)
train_features = train_features.fill_null(strategy='forward').fill_null(0)
print(f"Features prepared in {time.time() - start_time:.2f} seconds")

# Prepare data
start_time = time.time()
print("Preparing data...")
X = train_features.to_numpy()
y = train.select([col for col in train.columns if col.startswith('responder_')]).to_numpy()  # Use all responder columns
print(f"Data prepared in {time.time() - start_time:.2f} seconds")

# Define sequence length
seq_len = 5

# Custom Dataset Class for Lazy Sequence Generation
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, seq_len):
        self.X = X
        self.y = y
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        # Get sequence of features
        X_seq = self.X[idx:idx + self.seq_len]
        # Get the target for the end of the sequence
        y_seq = self.y[idx + self.seq_len]
        # Convert to tensor
        X_tensor = torch.tensor(X_seq, dtype=torch.float32)
        y_tensor = torch.tensor(y_seq, dtype=torch.float32)
        return X_tensor, y_tensor

# Create dataset and data loader
print('Create dataset and data loader')
dataset = TimeSeriesDataset(X, y, seq_len=seq_len)
batch_size = 64
start_time = time.time()
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print(f"Dataset and DataLoader created in {time.time() - start_time:.2f} seconds")

# Set up device for multi-GPU environment
print('Set up device for multi-GPU environment')
if torch.cuda.is_available():
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        device = torch.device("cuda")  # Use all available GPUs
    else:
        device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("Warning: CUDA is not available. Training will run on CPU.")

# Transformer Model Definition
class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model=64, nhead=2, num_layers=2, dropout=0.2, output_size=6):
        super(TransformerModel, self).__init__()
        self.input_projection = nn.Linear(input_size, d_model)
        self.pos_encoder = nn.Sequential(
            nn.Dropout(dropout)
        )
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )
        self.output_layer = nn.Linear(d_model, output_size)  # Output matches number of responder columns

    def forward(self, x):
        x = self.input_projection(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.output_layer(x[:, -1, :])  # Use the output from the last time step
        return x.squeeze()

# Instantiate the model
input_size = X.shape[1]  # Number of features per time step
output_size = y.shape[1]  # Number of responder columns
print('Create model')
start_time = time.time()
model = TransformerModel(input_size=input_size, d_model=64, nhead=2, num_layers=2, dropout=0.2, output_size=output_size)
print(f"Model created in {time.time() - start_time:.2f} seconds")

print('Move model to GPU')
# Set up DataParallel to use multiple GPUs
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model.to(device)

# Loss and optimizer
criterion = torch.nn.MSELoss()  # Base loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Custom weighted loss function to prioritize responder_6
def weighted_loss(outputs, targets):
    loss = F.mse_loss(outputs, targets, reduction='none')

    # Find the index of 'responder_6' dynamically
    responder_columns = [col for col in train.columns if col.startswith('responder_')]
    responder_6_index = responder_columns.index('responder_6')

    # Apply a higher weight for the 'responder_6' target column
    weights = torch.ones_like(targets).to(device)  # Ensure weights are on GPU
    responder_6_weight = 5.0  # Assign higher weight to responder_6
    weights[:, responder_6_index] = responder_6_weight

    weighted_loss = loss * weights
    return weighted_loss.mean()

checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
epochs = 50
for epoch in range(epochs):
    epoch_start_time = time.time()
    print(f"Start Epoch {epoch+1}/{epochs}")
    model.train()  # Set the model to training mode
    for batch_idx, (batch_X, batch_y) in enumerate(data_loader):
        batch_start_time = time.time()
        # Move batch data to the appropriate device
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)

        # Compute weighted loss
        loss = weighted_loss(outputs, batch_y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        if batch_idx % 3000 == 0:
          print(f"Batch {batch_idx + 1} completed in {time.time() - batch_start_time:.2f} seconds, Loss: {loss.item():.4f}")

    # Print epoch timing
    print(f"Epoch [{epoch+1}/{epochs}] completed in {time.time() - epoch_start_time:.2f} seconds, Loss: {loss.item():.4f}")

    # Save checkpoint after every epoch
    checkpoint_path = os.path.join(checkpoint_dir, f"transformer_checkpoint_epoch_{epoch+1}.pth")
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Checkpoint saved as {checkpoint_path}")

# Save the final trained model
torch.save(model.state_dict(), "transformer_model.pth")
print("Model saved as transformer_model.pth")


IndentationError: expected an indented block after 'if' statement on line 165 (<ipython-input-6-0b1b5bd2d077>, line 166)

In [ ]:
import matplotlib.pyplot as plt

# Load checkpoint for the model
checkpoint_path = "checkpoints/transformer_checkpoint_epoch_14.pth"
model.load_state_dict(torch.load(checkpoint_path))
model.to(device)

# Optimizer and Scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# Early Stopping Parameters
early_stopping_patience = 10
best_val_loss = float('inf')
patience_counter = 0

# Track Loss for Plotting
train_losses = []
val_losses = []

# Training loop with gradient clipping and validation
for epoch in range(15, epochs):
    epoch_start_time = time.time()
    print(f"Start Epoch {epoch+1}/{epochs}")

    # Training Phase
    model.train()
    running_loss = 0.0
    for batch_idx, (batch_X, batch_y) in enumerate(data_loader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()

        outputs = model(batch_X)
        loss = weighted_loss(outputs, batch_y)

        # Skip NaN losses
        if torch.isnan(loss):
            print(f"Skipping batch {batch_idx} due to NaN loss")
            continue

        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 3000 == 0:
            print(f"Batch {batch_idx + 1} completed, Loss: {loss.item():.4f}")

    train_loss = running_loss / len(data_loader)
    train_losses.append(train_loss)

    # Validation Phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_batch_X, val_batch_y in val_loader:  # Assuming val_loader is defined
            val_batch_X, val_batch_y = val_batch_X.to(device), val_batch_y.to(device)
            val_outputs = model(val_batch_X)
            val_loss += weighted_loss(val_outputs, val_batch_y).item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    print(f"Epoch [{epoch+1}/{epochs}] completed in {time.time() - epoch_start_time:.2f} seconds")
    print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Update Scheduler
    scheduler.step(val_loss)

    # Early Stopping Check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Save the best model
        torch.save(model.state_dict(), "best_transformer_model.pth")
        print("Best model saved as best_transformer_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

    # Save checkpoint after each epoch
    torch.save(model.state_dict(), f"checkpoints/transformer_checkpoint_restart_epoch_{epoch+1}.pth")
    print(f"Checkpoint saved as checkpoints/transformer_checkpoint_restart_epoch_{epoch+1}.pth")

# Plot Losses
plt.figure()
plt.plot(range(1, len(train_losses) + 1), train_losses, label="Training Loss")
plt.plot(range(1, len(val_losses) + 1), val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()


In [ ]:
if run_type == 'Transformer':
    # 模型参数
    input_size = 79
    d_model = 512
    nhead = 8
    num_layers = 3
    dropout = 0.1

    # 初始化模型
    model = TransformerModel(
        input_size=input_size,
        d_model=d_model,
        nhead=nhead,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)

    print_model_size(model)

    # 优化器设置
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
    loss_function = nn.MSELoss(reduction='none')

    # 训练循环
    epochs = 20
    best = float('-inf')
    degraded = 0
    best_model = model

    for epoch in range(epochs):
        train_loss, train_mse, train_r2 = train_model(
            model, train_loader, optimizer, loss_function, device
        )

        scheduler.step()
        val_mse, val_r2 = evaluate_model(model, val_loader)

        print(f'epoch {epoch}:')
        print(f'train loss {train_loss:.4f}, train_r2 {train_r2:.4f}, '
              f'train_mse {train_mse:.4f}')
        print(f'val_mse {val_mse:.4f}, val_r2 {val_r2:.4f}')
        print(f'lr: {scheduler.get_last_lr()[0]:.6f}')

        if val_r2 > best:
            best = val_r2
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), f'./model/js_{run_type}_unnorm.pth')
            degraded = 0
        else:
            degraded += 1

        if degraded > 10:
            print("Early stopping triggered")
            break

        torch.cuda.empty_cache()

    model = best_model
    test_mse, test_r2 = evaluate_model(model, test_loader)
    print(f'test R2 score is {test_r2}')


In [ ]:
lags_ : pl.DataFrame | None = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make predictions using either LSTM or Transformer model"""
    global lags_
    if lags is not None:
        lags_ = lags

    # 初始化预测DataFrame
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    # 选择特征列
    feature_cols = [f"feature_{i:02d}" for i in range(79)]

    # 处理测试数据
    test_features = test.select(feature_cols)
    test_features = test_features.fill_null(strategy='forward').fill_null(0)

    # 转换为tensor
    X_test = test_features.to_numpy()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

    # 进行预测
    model.eval()
    with torch.no_grad():
        pred = model(X_test_tensor).cpu().numpy()

    # 更新预测结果
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))

    # 验证输出格式
    assert isinstance(predictions, (pl.DataFrame, pd.DataFrame))
    assert list(predictions.columns) == ['row_id', 'responder_6']
    assert len(predictions) == len(test)
    print(predictions)
    return predictions

In [ ]:
import os

import pandas as pd
import polars as pl

import kaggle_evaluation.jane_street_inference_server


inference_server = kaggle_evaluation \
                        .jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )
